In [ ]:
!pip install -U spacy
!pip install spacy_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.8.4'

In [ ]:
!nvidia-smi

Thu Jan 23 13:46:18 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
cv_data=json.load(open('/content/drive/MyDrive/ResumeParser/dataset/resumedataset.json','r'))

In [ ]:
len(cv_data)

1012

In [ ]:
cv_data[230]

['\xa0 \xa0\nContact\nisaiascerqueiraribeiro@gmail.com\nwww.linkedin.com/in/isaiascribeiro\n(LinkedIn)\nisaiasribeirosec.cf/  (Personal)\nTop Skills\nethical hacking\npentest\ncybersecurity\nCertifications\nCloud Security\n2 - PrivacyTraining\nArchitecting with Google Compute\nengine\nInformation security and hacking\ntwo\nZERO TRUST Basic Certification\nIsaías Cerqueira Ribeiro\nInformation security analyst | Cyber Security | PenTest | Red Team |\nCCNA1 | CTF Play | Developer | CyberOps Associate (In progress)\nSão Gonçalo, Rio de Janeiro, Brasil\nSummary\nExperiência em trabalho de equipe nos eventos em Niterói,\nprojeto de rede local, apresentado na Universidade Anhanquera,\nonde foi possível adquirir aprendizados, conhecimentos da área,\ntrabalhar em grupo.Acredito que devemos estar sempre em\nconstante aprendizado, por isso priorizo os estudos e faço cursos\ncomplementares em TI, como o de Cyber Security na Cisco,\nDesenvolvedor Full Stack da Danki Code e outros cursos de\ndesenvo

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/ResumeParser/config/base_config.cfg /content/drive/MyDrive/ResumeParser/config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/ResumeParser/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
         # Check if annot is a dictionary and contains the 'entities' key
        if isinstance(annot, dict) and 'entities' in annot:
            annot = annot['entities']
        else:
            # Handle the case where annot is not a dictionary or doesn't have 'entities'
            print(f"Skipping item: annot is not in the expected format. Type: {type(annot)}, Value: {annot}")
             # Print a message for debugging
            continue  # Skip to the next item in the loop


        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            # Track indices
            entity_indices += list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except Exception as e:
                continue

            if span is None:

                err_data=str([start,end])+"   "+str(text)+"\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test =train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)

(708, 304)

In [ ]:
file = open('/content/drive/MyDrive/ResumeParser/model/train_file.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/ResumeParser/model/train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('/content/drive/MyDrive/ResumeParser/model/test_data.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 304/304 [00:02<00:00, 137.86it/s]


In [ ]:
!python -m spacy train /content/drive/MyDrive/ResumeParser/config/config.cfg --output/content/drive/MyDrive/ResumeParser/model/output--paths.train  /content/drive/MyDrive/ResumeParser/model/train_data.spacy --paths.dev/content/drive/MyDrive/ResumeParser/model/test_data.spacy --gpu-id 0

✔ Created output directory:
/content/drive/MyDrive/ResumeParser/model/output
ℹ Saving to output directory:
/content/drive/MyDrive/ResumeParser/model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `forc

In [ ]:
nlp=spacy.load('/content/drive/MyDrive/ResumeParser/model/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.

In [ ]:
!pip install pymupdf

In [ ]:
import sys
import fitz  # PyMuPDF library

fname = '/content/drive/MyDrive/ResumeParser/test/CV_Anupa.pdf'
doc = fitz.open(fname)


In [ ]:
text=" "
for page in doc:
  text=text+ str(page.get_text())


In [ ]:
text

' Anupa Gaire \n \nUndergraduate Student of Computer Engineering   \n \nKhwopa Engineering College, Libali -8, Bhaktapur, Nepal  \n \n \nContact Information: \n \nPhone No \n:  \n(+977) 9860077724 (Cell) \nEmail \n:  \naanupa.gaire@gmail.com, 770304@khec.edu.np \n \nAddress \n: \nBagmati Province, Suryavinayak-05, Katunje, Bhaktapur, \nNepal  \nGitHub  \n: \nanupagaire \nLinkedIn \n: \nanupagaire \nWebsite \n: \nanupagaire.com.np  \nDate of Birth \n: \n07 November 2003 \n \nEducation: \n \nCurrent \n7th semester,BE Computer Engineering, Khwopa Engineering College, \nBhaktapur, Nepal. \n2020 \nPlus Two (+2) in Science ,Kathmandu Model College, Bagbazar, Kathmandu, \nNepal. \n2018 \nSecondary Education Examination (SEE),Om Secondary School, \nBhaktapur, Nepal \n \nWork Experience: \n \n• Internship at ……. \n \nTechnical Skills: \n \n• \nText formatting and office computing: MS Word, Power Point, MS Excel \n• Software: MATLAB (Basic), LaTeX (Basic) \n• \nWeb Development: WordPress, Flask 

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, " ->>>>",ent.label_)

Anupa Gaire  ->>>> NAME
Khwopa Engineering College, Libali -8, Bhaktapur, Nepal  ->>>> COLLEGE NAME
Current  ->>>> UNIVERSITY
BE Computer Engineering  ->>>> DEGREE
Khwopa Engineering College  ->>>> COLLEGE NAME
WordPress  ->>>> SKILLS
Flask (Basic)  ->>>> SKILLS
MySQL  ->>>> SKILLS
HTML  ->>>> SKILLS
CSS  ->>>> SKILLS
C  ->>>> SKILLS
C++  ->>>> SKILLS
JavaScript  ->>>> SKILLS
PHP  ->>>> SKILLS
Python (Basic)  ->>>> SKILLS
Khwopa Engineering College  ->>>> COLLEGE NAME
Er. Bikash Chawal  ->>>> UNIVERSITY
Khwopa Engineering College  ->>>> COLLEGE NAME
